In [2]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import xmltodict
import re

urls = ["https://hebbarskitchen.com/post-sitemap1.xml", "https://hebbarskitchen.com/post-sitemap2.xml"]
urldict = [] # To maintain a list of available recipe urls in the site
recipedict = [] # To maintain a recipe-dictionary

# Extract the available recipe urls in the site
for url in urls:

    response = requests.get(url)
    xml = response.text
    soup = BeautifulSoup(xml)
    
    urlTags = soup.find_all("url")
    
    for urlTag in urlTags:
        urldict.append(urlTag.findNext("loc").text)

# Build the recipe-dictionary with values         
for url in urldict:

    if re.match("^(https):\/\/[a-z]*\.(com)\/[a-z,A-Z,0-9,-]*\/$", url) != None:
        response = requests.get(url)
        soup = BeautifulSoup(response.text)

        if (soup.find("span", class_="wprm-recipe-course")!= None) and (soup.find("span", class_="wprm-recipe-cuisine") != None):
            
            ingredientList = [] # To maintain a recipe wise ingredient list
            ingredients = soup.find_all("span", class_="wprm-recipe-ingredient-name")
            
            for ingredient in ingredients: 
                    ingredientList.append(ingredient.text)
        
            recipedict.append(
                [soup.find("h2", class_="wprm-recipe-name").text,
                 url,
                 soup.find("span", class_="wprm-recipe-course").text,
                 soup.find("span", class_="wprm-recipe-cuisine").text,
                 ingredientList
                ])
        
len(recipedict)

1198

In [3]:
df = pd.DataFrame(recipedict, columns=["Name","URL","Course","Cuisine","ingredients"])
# df.to_json(r'Updated_HebbarsRecipes.json', orient='index')